In [1]:
import pandas as pd
import numpy as np

queries = ('q11', 'q12', 'q13', 'q21', 'q22', 'q23', 'q31', 'q32', 'q33', 'q34', 'q41', 'q42', 'q43')

def load_performance_data(query):
    return pd.read_csv(f'{query}-performance.log', header=None, names=['execution_time_ms'])

In [2]:
results = pd.DataFrame()
for q in queries:
    data = load_performance_data(q)
    results[q] = data['execution_time_ms']
    
results.describe()

,q11,q12,q13,q21,q22,q23,q31,q32,q33,q34,q41,q42,q43
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,13.066991,14.023130,13.997501,12.816319,10.199652,8.615798,15.705098,10.701047,9.606535,9.371616,18.321657,16.503086,14.213607
std,0.058390,0.072740,0.269677,0.706162,0.794067,0.724450,0.890415,0.879745,0.910551,0.871409,0.587261,1.104309,1.048972
min,13.007900,13.943600,13.896500,12.214700,9.589190,8.022270,14.961100,9.761060,8.842150,8.684150,17.310000,14.834000,12.864300
25%,13.040025,13.989675,13.932225,12.332725,9.676840,8.124688,15.110475,10.073450,8.953672,8.766238,17.565275,15.571700,13.154550
50%,13.056750,14.008150,13.948000,12.418550,9.771410,8.166710,15.192350,10.149700,9.054955,8.840790,18.639900,16.354650,14.339450
75%,13.074275,14.038375,13.974925,13.328025,10.628425,9.132352,16.318775,11.451850,10.217500,10.008230,18.675475,17.344150,14.808900
max,13.519900,14.484600,16.478100,14.455200,13.566200,11.105900,19.137400,13.212700,12.491800,12.369300,20.065800,19.253400,17.193600


In [3]:
results.head(50).describe()

,q11,q12,q13,q21,q22,q23,q31,q32,q33,q34,q41,q42,q43
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,13.056242,14.010114,13.957900,12.703756,10.112584,8.490447,15.556806,10.618486,9.503685,9.261065,18.303060,16.417116,14.050908
std,0.036912,0.069158,0.062853,0.561501,0.772554,0.542437,0.610779,0.719317,0.778629,0.720756,0.642711,1.069227,0.805167
min,13.007900,13.943600,13.896500,12.226100,9.589190,8.022270,14.961100,10.025000,8.842150,8.684150,17.310000,14.834000,12.864300
25%,13.038825,13.978875,13.926375,12.318025,9.663770,8.113590,15.108825,10.057225,8.929253,8.755852,17.537900,15.495775,13.105425
50%,13.044600,14.001750,13.943350,12.390200,9.728450,8.152540,15.173850,10.167450,9.047495,8.818255,18.616100,16.370150,14.339450
75%,13.066275,14.020075,13.963825,13.192050,10.482150,8.963242,16.058700,11.320775,10.121825,9.868580,18.663100,17.187275,14.496925
max,13.245000,14.454100,14.301200,14.063800,13.566200,9.559720,16.895000,12.982800,11.750200,11.669000,20.065800,19.253400,15.862300


In [4]:
results[50:].describe()

,q11,q12,q13,q21,q22,q23,q31,q32,q33,q34,q41,q42,q43
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,13.077740,14.036146,14.037102,12.928882,10.286721,8.741150,15.853390,10.783607,9.709384,9.482168,18.340254,16.589056,14.376306
std,0.072732,0.074577,0.373878,0.816312,0.813414,0.856773,1.088085,1.016055,1.023365,0.994874,0.531982,1.142609,1.233217
min,13.016400,13.966900,13.906800,12.214700,9.615890,8.088460,15.010200,9.761060,8.918110,8.719850,17.385700,14.898700,12.974500
25%,13.048525,13.996525,13.937900,12.348225,9.709202,8.138368,15.124750,10.075725,8.976318,8.782905,17.746550,15.708075,13.242125
50%,13.064500,14.025900,13.956700,12.445200,9.806265,8.206130,15.199350,10.142050,9.064305,8.860565,18.649800,16.353150,14.288000
75%,13.077450,14.047300,13.985700,13.906650,11.204850,9.726675,16.774025,11.762900,10.618800,10.480425,18.686950,17.400225,15.365700
max,13.519900,14.484600,16.478100,14.455200,12.197700,11.105900,19.137400,13.212700,12.491800,12.369300,18.777700,18.899400,17.193600
